In [32]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import MemorySaver

In [33]:
load_dotenv()
llm = ChatOpenAI()

In [34]:
class jokeState(TypedDict):
    topic: str 
    joke: str 
    explanation: str 

In [35]:
def generate_joke(state: jokeState):
    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [36]:
def generate_explanation(state: jokeState):
    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [37]:
state = StateGraph(jokeState)

state.add_node('generate_joke', generate_joke)
state.add_node('generate_explanation', generate_explanation)

state.add_edge(START, 'generate_joke')
state.add_edge('generate_joke', 'generate_explanation')
state.add_edge('generate_explanation', END)

checkpointer = MemorySaver()

workflow = state.compile(checkpointer=checkpointer)

In [38]:
initial_state = {
    'topic': "Create a joke about monkey"
}

workflow.invoke(initial_state)

ValueError: Checkpointer requires one or more of the following 'configurable' keys: thread_id, checkpoint_ns, checkpoint_id